# Set up

In [ ]:
!pip install albumentations
!pip install opencv-python
!pip install ultralytics
!pip install roboflow
!pip install inference supervision

In [1]:
import os, shutil
from os import listdir
from os.path import isfile, join
import cv2
import albumentations as A
import copy
import random
import numpy as np
from ultralytics import YOLO
import torch
import shutil

In [2]:
# If you don't have yolov9 set up
!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q

Cloning into 'yolov9'...
remote: Enumerating objects: 325, done.
remote: Total 325 (delta 0), reused 0 (delta 0), pack-reused 325
Receiving objects: 100% (325/325), 2.25 MiB | 2.23 MiB/s, done.
Resolving deltas: 100% (164/164), done.
/home/potenza/Ortho-AUV/yolov9
/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


In [ ]:
# If you already have yolov9 in your repo
%cd yolov9

In [3]:
from roboflow import Roboflow

rf = Roboflow(api_key="HV9jB1dcJZHZd1YO5S7C") 
project = rf.workspace("comp400").project("comp400-fish-detection")
dataset = project.version(3).download("yolov9")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to COMP400-Fish-detection-3 in yolov9:: 100%|██████████| 90/90 [00:00<00:00, 7058.35it/s]


# Train (training with on roboflow's website is easier - ignore this section)

In [4]:
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

In [7]:
!python3 train.py \
     --data COMP400-Fish-detection-3/data.yaml \
     --weights weights/gelan-c.pt \
     --cfg models/detect/gelan-c.yaml \
     --hyp hyp.scratch-high.yaml \
     --device cpu
     

train: weights=weights/gelan-c.pt, cfg=models/detect/gelan-c.yaml, data=COMP400-Fish-detection-3/data.yaml, hyp=hyp.scratch-high.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=cpu, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, flat_cos_lr=False, fixed_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, min_items=0, close_mosaic=0, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLOv5 🚀 1e33dbb Python-3.8.10 torch-2.2.1+cu121 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=7.5, cls=0.5, cls_pw=1.0, dfl=1.5, obj_pw=1.0, iou_t=0.2, anchor_t=5.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=

IOStream.flush timed out


# Deploy

In [2]:
from roboflow import Roboflow

rf = Roboflow(api_key='HV9jB1dcJZHZd1YO5S7C')
project = rf.workspace('comp400').project('comp400-fish-detection')
model = project.version(4).model

loading Roboflow workspace...
loading Roboflow project...


In [4]:
import os 

data_dir = "../data4"
file_name = "geo.txt"

with open(os.path.join(data_dir, file_name), "r") as f:
     lines = f.readlines()
     lines = lines[1:]  # Skip the header line

filenames_raw, filenames_depth = [], []

for line in lines:
     filename_raw, filename_depth, _, _, _, _, _, _ = line.strip().split()
     filenames_raw.append(filename_raw)
     filenames_depth.append(filename_depth)


In [5]:
!mkdir predictions

In [14]:
file = filenames_raw[0]
prediction = model.predict(os.path.join(data_dir, file), confidence=50, overlap=50).json()["predictions"][0]
x = prediction["x"]
print(prediction)

{'x': 342.5, 'y': 316.0, 'width': 79.0, 'height': 42.0, 'confidence': 0.8940266370773315, 'class': 'fish1', 'class_id': 0, 'detection_id': '069dbe43-9c98-4cab-a6d0-70ce6f37994c', 'image_path': '../data1/19:52:44.171.jpg', 'prediction_type': 'ObjectDetectionModel'}


In [5]:
with open("prediction.txt", "w") as f:
     f.write("filename_raw filename_depth x y width height")
     for i in range(len(filenames_raw)):
          prediction = model.predict(os.path.join(data_dir, filenames_raw[i]), confidence=50, overlap=50).json()["predictions"]
          for j in range(len(prediction)):
               instance = prediction[j]
               x = instance["x"]
               y = instance["y"]
               width = instance["width"]
               height = instance["height"]
               f.write(f"\n{filenames_raw[i]} {filenames_depth[i]} {x} {y} {width} {height}")
     
     # uncoment next line if you want to visualize predictions
     # 

In [6]:
model.predict(os.path.join(data_dir, "22:28:23.171_raw.jpg"), confidence=50, overlap=50).save(f"prediction_22:28:23.171_raw.jpg.jpg")